In [114]:
import json

from selenium import webdriver
import requests
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import pandas as pd
from pyspark.sql.types import *

In [84]:
options = webdriver.ChromeOptions()

options.set_capability(
    'goog:loggingPrefs', {"performance": "ALL", "browser": "ALL"}
)

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

In [102]:
driver.set_page_load_timeout(10)

try:
    driver.get("https://www.sofascore.com/newcastle-united-manchester-united/DYBe#id:12240571,tab:statistics")
except:
    pass
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [103]:
logs_raw = driver.get_log("performance")

In [104]:
logs = [json.loads(lr['message'])['message'] for lr in logs_raw]

In [105]:
logs[0]

{'method': 'Network.requestWillBeSent',
 'params': {'documentURL': 'https://www.sofascore.com/newcastle-united-manchester-united/DYBe',
  'frameId': '1DF68630F9CB7D99C9F847C877735757',
  'hasUserGesture': False,
  'initiator': {'stack': {'callFrames': [{'columnNumber': 197228,
      'functionName': '',
      'lineNumber': 0,
      'scriptId': '69',
      'url': 'https://www.sofascore.com/_next/static/chunks/9998.cf3432874fdfc511.js'},
     {'columnNumber': 767320,
      'functionName': '',
      'lineNumber': 0,
      'scriptId': '31',
      'url': 'https://www.sofascore.com/_next/static/chunks/pages/_app-0c11622be9c5e603.js'},
     {'columnNumber': 748903,
      'functionName': '',
      'lineNumber': 0,
      'scriptId': '31',
      'url': 'https://www.sofascore.com/_next/static/chunks/pages/_app-0c11622be9c5e603.js'},
     {'columnNumber': 749008,
      'functionName': '',
      'lineNumber': 0,
      'scriptId': '31',
      'url': 'https://www.sofascore.com/_next/static/chunks/page

In [107]:
for x in logs:
    if 'shotmap' in x['params'].get('headers', {}).get(':path', ''):
        print(x['params'].get('headers', {}).get(':path', ''))
        break

/api/v1/event/12240571/shotmap


In [108]:
x

{'method': 'Network.requestWillBeSentExtraInfo',
 'params': {'associatedCookies': [{'blockedReasons': [],
    'cookie': {'domain': '.sofascore.com',
     'expires': 1723749493,
     'httpOnly': False,
     'name': '_gcl_au',
     'path': '/',
     'priority': 'Medium',
     'sameParty': False,
     'secure': False,
     'session': False,
     'size': 31,
     'sourcePort': 443,
     'sourceScheme': 'Secure',
     'value': '1.1.443955146.1715973493'},
    'exemptionReason': 'None'},
   {'blockedReasons': [],
    'cookie': {'domain': '.sofascore.com',
     'expires': 1750533494.835247,
     'httpOnly': False,
     'name': '_ga',
     'path': '/',
     'priority': 'Medium',
     'sameParty': False,
     'secure': False,
     'session': False,
     'size': 30,
     'sourcePort': 443,
     'sourceScheme': 'Secure',
     'value': 'GA1.1.1229404817.1715973494'},
    'exemptionReason': 'None'},
   {'blockedReasons': [],
    'cookie': {'domain': '.sofascore.com',
     'expires': 1749842295,
   

In [112]:
shotmap = json.loads(driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': x['params']['requestId']})['body'])['shotmap']

WebDriverException: Message: unknown error: unhandled inspector error: {"code":-32000,"message":"No resource with given identifier found"}
  (Session info: chrome=124.0.6367.63)
Stacktrace:
	GetHandleVerifier [0x00B2C113+48259]
	(No symbol) [0x00ABCA41]
	(No symbol) [0x009B0A17]
	(No symbol) [0x0099C734]
	(No symbol) [0x0099B4DD]
	(No symbol) [0x0099B9DD]
	(No symbol) [0x0099B978]
	(No symbol) [0x009B36A4]
	(No symbol) [0x00A34331]
	(No symbol) [0x00A10D9C]
	(No symbol) [0x00A29B9C]
	(No symbol) [0x00A10B36]
	(No symbol) [0x009E570D]
	(No symbol) [0x009E62CD]
	GetHandleVerifier [0x00DE65A3+2908435]
	GetHandleVerifier [0x00E23BBB+3159851]
	GetHandleVerifier [0x00BC50CB+674875]
	GetHandleVerifier [0x00BCB28C+699900]
	(No symbol) [0x00AC6244]
	(No symbol) [0x00AC2298]
	(No symbol) [0x00AC242C]
	(No symbol) [0x00AB4BB0]
	BaseThreadInitThunk [0x76367BA9+25]
	RtlInitializeExceptionChain [0x772EBE3B+107]
	RtlClearBits [0x772EBDBF+191]


In [117]:
url = "https://www.sofascore.com/newcastle-united-manchester-united/DYBe#12240571"
response = requests.get(url)

ReadTimeout: HTTPSConnectionPool(host='www.sofascore.com', port=443): Read timed out. (read timeout=None)